In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.svm import SVR
from sklearn.metrics import r2_score, mean_absolute_error

import warnings
warnings.filterwarnings('ignore')

## Loading a DataFrame

In [2]:
df = sns.load_dataset('tips')
df.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 244 entries, 0 to 243
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   total_bill  244 non-null    float64 
 1   tip         244 non-null    float64 
 2   sex         244 non-null    category
 3   smoker      244 non-null    category
 4   day         244 non-null    category
 5   time        244 non-null    category
 6   size        244 non-null    int64   
dtypes: category(4), float64(2), int64(1)
memory usage: 7.4 KB


In [4]:
df.isnull().sum()

total_bill    0
tip           0
sex           0
smoker        0
day           0
time          0
size          0
dtype: int64

In [5]:
df['sex'].value_counts()

sex
Male      157
Female     87
Name: count, dtype: int64

In [6]:
df['smoker'].value_counts()

smoker
No     151
Yes     93
Name: count, dtype: int64

In [7]:
df['day'].value_counts()

day
Sat     87
Sun     76
Thur    62
Fri     19
Name: count, dtype: int64

In [8]:
df['time'].value_counts()

time
Dinner    176
Lunch      68
Name: count, dtype: int64

## Train Test Split

In [9]:
X = df.drop('total_bill', axis=1)
y = df['total_bill']

In [10]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=10
)

## Feature Encoding

In [11]:
df.columns

Index(['total_bill', 'tip', 'sex', 'smoker', 'day', 'time', 'size'], dtype='object')

### Label Encoding

In [12]:
sex_label_encoder = LabelEncoder()
smoker_label_encoder = LabelEncoder()
time_label_encoder = LabelEncoder()

X_train['sex'] = sex_label_encoder.fit_transform(X_train['sex'])
X_train['smoker'] = smoker_label_encoder.fit_transform(X_train['smoker'])
X_train['time'] = time_label_encoder.fit_transform(X_train['time'])

In [13]:
X_train.head()

,tip,sex,smoker,day,time,size
58,1.76,1,1,Sat,0,2
1,1.66,1,0,Sun,0,3
2,3.50,1,0,Sun,0,3
68,2.01,1,0,Sat,0,2
184,3.00,1,1,Sun,0,2


In [14]:
X_test['sex'] = sex_label_encoder.transform(X_test['sex'])
X_test['smoker'] = smoker_label_encoder.transform(X_test['smoker'])
X_test['time'] = time_label_encoder.transform(X_test['time'])

### One Hot Encoding

In [15]:
column_transformer = ColumnTransformer(
    transformers=[('onehot', OneHotEncoder(drop='first'), [3])],
    remainder='passthrough'
)

X_train = column_transformer.fit_transform(X_train)
X_test = column_transformer.transform(X_test)

## Model Training

In [16]:
model = SVR(kernel="rbf")
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

In [17]:
r2 = r2_score(y_true=y_test, y_pred=y_pred)
mae = mean_absolute_error(y_true=y_test, y_pred=y_pred)

print(f"R2 Score: {r2}")
print(f"Mean Absolute Error: {mae}")

R2 Score: 0.46028114561159283
Mean Absolute Error: 4.1486423210190235


## Hyperparameter Tuning

In [18]:
model = SVR()
param_grid = {
    'C': [0.1, 1, 10, 100, 1000],
    'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
    'kernel': ['rbf']
}

grid_model = GridSearchCV(
    estimator=model, param_grid=param_grid,
    refit = True, verbose = 3
)
grid_model.fit(X_train, y_train)

Fitting 5 folds for each of 25 candidates, totalling 125 fits
[CV 1/5] END .......C=0.1, gamma=1, kernel=rbf;, score=-0.067 total time=   0.0s
[CV 2/5] END .......C=0.1, gamma=1, kernel=rbf;, score=-0.058 total time=   0.0s
[CV 3/5] END .......C=0.1, gamma=1, kernel=rbf;, score=-0.145 total time=   0.0s
[CV 4/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.025 total time=   0.0s
[CV 5/5] END .......C=0.1, gamma=1, kernel=rbf;, score=-0.089 total time=   0.0s
[CV 1/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.013 total time=   0.0s
[CV 2/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.021 total time=   0.0s
[CV 3/5] END .....C=0.1, gamma=0.1, kernel=rbf;, score=-0.010 total time=   0.0s
[CV 4/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.124 total time=   0.0s
[CV 5/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.050 total time=   0.0s
[CV 1/5] END ....C=0.1, gamma=0.01, kernel=rbf;, score=-0.053 total time=   0.0s
[CV 2/5] END ....C=0.1, gamma=0.01, kernel=rbf;

GridSearchCV(estimator=SVR(),
             param_grid={'C': [0.1, 1, 10, 100, 1000],
                         'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
                         'kernel': ['rbf']},
             verbose=3)

In [19]:
grid_model.best_params_

{'C': 1000, 'gamma': 0.0001, 'kernel': 'rbf'}

In [20]:
grid_prediction = grid_model.predict(X_test)

In [21]:
r2 = r2_score(y_true=y_test, y_pred=grid_prediction)
mae = mean_absolute_error(y_true=y_test, y_pred=grid_prediction)

print(f"R2 Score: {r2}")
print(f"Mean Absolute Error: {mae}")

R2 Score: 0.5081618245078687
Mean Absolute Error: 3.8685147526100234
